In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy
from scipy.stats import norm

In [ ]:
pd.options.display.max_columns = None
pd.options.display.max_rows = None
df = pd.read_csv("src/listings_summary.csv")

In [ ]:
#source : https://www.kaggle.com/mhmdpkts/predict-price-ann-data-cleaning-future-engineering
fig, ax = plt.subplots(figsize = (20, 5))
sns.heatmap(df.isna(), cmap = "cubehelix_r", yticklabels='')
plt.show()

In [ ]:
#Sort out the Price-Section
df.price = df.price.str.replace('$', '').str.replace(',', '').astype(float)
df.cleaning_fee = df.cleaning_fee.str.replace('$', '').str.replace(',', '').astype(float)
df.security_deposit = df.security_deposit.str.replace('$', '').str.replace(',', '').astype(float)
df.extra_people = df.extra_people.str.replace('$', '').str.replace(',', '').astype(float)

df['cleaning_fee'].fillna(0, inplace=True)
df['security_deposit'].fillna(0, inplace=True)
#  + df['cleaning_fee'] + df['security_deposit']
df['total_price'] = df['price']
df.drop(['price'], inplace=True, axis = 1)

In [ ]:
def nan_checker(df):
    df_nan = pd.DataFrame([[var, df[var].isna().sum() / df.shape[0], df[var].dtype]
                           for var in df.columns if df[var].isna().sum() > 0],
                          columns=['var', 'proportion', 'dtype'])
    return df_nan

In [ ]:
#removing 100% of the NULL values and identity values
col_list_to_drop = [] 
columns = df.columns
df_nan = nan_checker(df)
length = len(df)
for col in df.columns:  
    if (df[col].dtype != 'float64'and df[col].nunique(dropna=True) == df[col].notnull().sum()):
        col_list_to_drop.append(col)
df.drop(col_list_to_drop, inplace=True, axis = 1)

In [ ]:
for row in df_nan.iterrows():
    column_name = row[1]['var']
    rate = round(row[1]['proportion'], 3)
    if rate > 0.5:
        df.drop(column_name, inplace=True, axis = 1)

In [ ]:
df.describe(include="all")

We will not need columns containing the following types of data-for our analysis


columns with only true/false values
columns that have text (not within the scope of our analysis)
columns containing personnel information, except for the ones that guranntee some kind of assurance(licence, or profile pictures)

Make everything as relevant, unless it is really irrelevant to your prediction.

In [ ]:
review_host_text_columns =["scrape_id",
                           "picture_url",
                           "last_scraped",
                           "name",
                           "summary", 
                           "space",
                           "description",
                           "experiences_offered", 
                           "neighborhood_overview",
                           "transit",
                           "house_rules", 
                           "host_id",
                           "host_url",
                           "host_location",
                           "host_neighbourhood",
                           "host_name",
                           "host_since", 
                           "host_thumbnail_url",
                           "host_picture_url",
                           "host_listings_count",
                           "host_total_listings_count", 
                           "host_verifications",
                           "calendar_updated",
                           "has_availability",
                           "availability_30",
                           "availability_60",
                           "availability_90",
                           "calendar_last_scraped",
                           "number_of_reviews",
                           "first_review",
                           "last_review",
                           "review_scores_rating",
                           "review_scores_accuracy",
                           "review_scores_cleanliness",
                           "review_scores_checkin",
                           "review_scores_communication",
                           "review_scores_location",
                           "review_scores_value",
                           "calculated_host_listings_count", 
                           "reviews_per_month"]

df_dropped = df.drop(review_host_text_columns, inplace=False, axis = 1).copy(deep=True)
df_dropped.describe(include="all")

<h3>Work on the Missing Data</h3>

 1) Strategy 1: Drop all missing rows and check the outcome
 
 2) Strategy 2: Impute the missing values with mean or most repetative element


In [ ]:
#Strategy 1 
df_dropped_rows = df_dropped.dropna()
percentange_of_rows_dropped = (len(df_dropped) - len(df_dropped_rows))/len(df_dropped)
print("Percentage of Rows Dropped  {}".format(percentange_of_rows_dropped * 100))

In [ ]:
df_nan = nan_checker(df_dropped_rows)
df_nan

In [ ]:
#strategy 2 Imputing all the values
from sklearn.impute import SimpleImputer
#filling all the missing values with mean
numerical_imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
category__imputer = SimpleImputer(missing_values=np.nan,strategy="most_frequent")


In [ ]:
df_dropped_imputed = df_dropped_rows.copy(deep=True)
df_nan = nan_checker(df_dropped_imputed)
# df_nan

In [ ]:
for row in df_nan.iterrows():
    column_name = row[1]['var']
    if df_dropped_imputed[column_name].dtype == 'float64':
        df_dropped_imputed[column_name] = numerical_imputer.fit_transform(df_dropped_imputed[[column_name]])
    else:
        df_dropped_imputed[column_name] = category__imputer.fit_transform(df_dropped_imputed[[column_name]])

In [ ]:
df_nan = nan_checker(df_dropped_imputed)
df_nan

<h1> Null Data Removed for all columns </h1>

In [ ]:
#decide between strategy 1 or 2
df_na_filtered = df_dropped_imputed.copy(deep=True)

In [ ]:
df_na_filtered.describe(include="all")

Dropping all information related to location, except for neighbour_group_cleansed: This column indicates which part of berlin the listings are located in. Secondly, the lattitude and logitude would be converted to distance from centre.

In [ ]:
location_columns                 = ["street",
                                   "neighbourhood",
                                   "neighbourhood_cleansed",
                                   "neighbourhood_group_cleansed",
                                   "city",
                                   "state",
                                   "zipcode",
                                   "market",
                                   "smart_location",
                                   "country_code",
                                   "country",
                                   "is_location_exact",
                                   ]
# for col in location_columns:
#     print("------------------Column Name: {}\n{}".format(col, df_na_filtered[col].value_counts()))

Location Columns having highly skewed values will be removed, todo: need a visual aid of all the columns and count of each value (except for latitude and longitude)

In [ ]:
df_na_filtered.drop(location_columns, axis=1, inplace=True)

In [ ]:
df_na_filtered.head()

In [ ]:
#Using the Lattitue and Logitude to convert to a distance from the city center,

#source : https://www.kaggle.com/mhmdpkts/predict-price-ann-data-cleaning-future-engineering
#create distance feature from latitude and longtitude 
from geopy.distance import great_circle
def distance_from_berlin(lat, lon):
    berlin_centre = (52.5027778, 13.404166666666667)
    record = (lat, lon)
    #returns surface distance in kilometers
    return great_circle(berlin_centre, record).km

#add distanse dataset
df_na_filtered['distance'] = df_na_filtered.apply(lambda x: distance_from_berlin(x.latitude, x.longitude), axis=1)
df_na_filtered.drop(['latitude', 'longitude'], axis=1, inplace=True)
df_na_filtered.head()

In [ ]:
#Converting the boolean to 0 or 1 value,
boolean_mapping =  {
    'True':1, 'False':0,
    't':1, 'f':0, 
    'T':1, 'F':0, 
     0:0, 1:1
}

boolean_features = ["host_is_superhost",
                    "host_has_profile_pic",
                    "host_identity_verified",
                    "requires_license",
                    "instant_bookable",
                    "is_business_travel_ready",
                    "require_guest_profile_picture",
                    "require_guest_phone_verification"]

for col in boolean_features:
    df_na_filtered[col]= df_na_filtered[col].map(boolean_mapping)
    
df_na_filtered.head()

In [ ]:
#Converting ammenities to a count
#amenities count
def amenities_counter(x):
    return len(x.split(","))


df_na_filtered["amenities_count"] = df_na_filtered.amenities.map(amenities_counter)
df_na_filtered['Laptop_friendly_workspace'] = df_na_filtered['amenities'].str.contains('Laptop friendly workspace')
df_na_filtered['TV'] = df_na_filtered['amenities'].str.contains('TV')
df_na_filtered['Family_kid_friendly'] = df_na_filtered['amenities'].str.contains('Family/kid friendly')
df_na_filtered['Host_greets_you'] = df_na_filtered['amenities'].str.contains('Host greets you')
df_na_filtered['Smoking_allowed'] = df_na_filtered['amenities'].str.contains('Smoking allowed')
df_na_filtered['Hot_water'] = df_na_filtered['amenities'].str.contains('Hot water')
df_na_filtered['Fridge'] = df_na_filtered['amenities'].str.contains('Refrigerator')

df_na_filtered['Laptop_friendly_workspace'] = df_na_filtered['Laptop_friendly_workspace'].map(boolean_mapping)
df_na_filtered['TV'] = df_na_filtered['TV'].map(boolean_mapping)
df_na_filtered['Family_kid_friendly'] = df_na_filtered['Family_kid_friendly'].map(boolean_mapping)
df_na_filtered['Host_greets_you'] = df_na_filtered['Host_greets_you'].map(boolean_mapping)
df_na_filtered['Smoking_allowed'] = df_na_filtered['Smoking_allowed'].map(boolean_mapping)
df_na_filtered['Hot_water'] = df_na_filtered['Hot_water'].map(boolean_mapping)
df_na_filtered['Fridge'] = df_na_filtered['Fridge'].map(boolean_mapping)

amenities_columns = ['Laptop_friendly_workspace', 'TV', 'Family_kid_friendly', 'Host_greets_you', 
                     'Smoking_allowed', 'Hot_water', 'Fridge']



#Drop the amenities column from prediction data-frame
df_na_filtered.drop(['amenities'], axis=1, inplace=True)

In [ ]:
df_na_filtered.head()

Choose either to include individual Ammenities or just the count of ammenities in prediction, by concatinating the values

We will reoriganize the property_type in to smaller set of types : 

In [ ]:
#Property Types:
property_mapping = {'Guesthouse':'Guesthouse', 
                    'Apartment':'Apartment', 
                    'Condominium':'Apartment', 
                    'Loft':'others',
                    'House':'Apartment',
                    'Serviced apartment':'Apartment',
                    'Townhouse':'Apartment',
                    'Boutique hotel':'Hotel',
                    'Bed and breakfast':'Hotel',
                    'Bungalow':'Apartment',
                    'Boat':'others',
                    'Guest suite':'Guesthouse',
                    'Other':'others',
                    'Island':'others',
                    'Hostel':'others',
                    'Train':'others',
                    'Camper/RV':'others',
                    'Houseboat':'others',
                    'Cabin':'others',
                    'Cottage':'Apartment',
                    'Tiny house':'Apartment',
                    'Chalet':'others',
                    'Castle':'others',
                    'Villa':'Apartment',
                    'Aparthotel':'others',
                    'Hotel':'Hotel',
                    'Tipi':'others',
                    'Resort':'Hotel',
                    'In-law':'others',
                    'Cave':'others',
                    'Barn':'others',
                    'Pension (South Korea)':'others',
                    'Guesthouse':'Guesthouse',
                    'Casa particular (Cuba)':'others',
                    'others':'others'}
df_na_filtered['property_type']=df_na_filtered['property_type'].map(property_mapping)
# sns.set(style="darkgrid")
# fig, ax = plt.subplots()
# sns.countplot(x="property_type", data=df_na_filtered)
# sns.despine()

In [ ]:
# #Trying the same with Labelencoder
# from sklearn.preprocessing import LabelEncoder# creating initial dataframe
# labelencoder = LabelEncoder()
# df_na_filtered['property_type'] = labelencoder.fit_transform(df_na_filtered['property_type'])
# df_na_filtered.head()

In [ ]:
# sns.set(style="darkgrid")
# fig, ax = plt.subplots()
# fig.set_size_inches(20, 5)
# sns.countplot(x="neighbourhood_group_cleansed", data=df_na_filtered)
# sns.despine()

In [ ]:
sns.set(style="darkgrid")
fig, ax = plt.subplots()
sns.countplot(x="room_type", data=df_na_filtered)
sns.despine()

In [ ]:
#Drop or include the room_type
#df_na_filtered.drop(['room_type'], inplace=True, axis=1)
#df_na_filtered.head()

In [ ]:
df_na_filtered["bed_type"].unique()
bed_mapping =  {'Real Bed':'Real Bed', 
                'Others':'Others',
                'Pull-out Sofa': 'Others',
                'Couch':'Others',
                'Futon':'Others',
                'Airbed':'Others'}
df_na_filtered['bed_type']=df_na_filtered['bed_type'].map(bed_mapping)
df_na_filtered.head()

In [ ]:
df_na_filtered["cancellation_policy"].unique()
cancellation_mapping =  {'strict_14_with_grace_period':'strict', 
                'flexible':'flexible',
                'moderate': 'moderate',
                'super_strict_30':'strict',
                'super_strict_60':'strict',
                'strict':'strict'}
df_na_filtered['cancellation_policy']=df_na_filtered['cancellation_policy'].map(cancellation_mapping)
df_na_filtered.head()

In [ ]:
#Generate a Categorical and Numerical Column List
numerical_columns = []
categoical_columns = []
for column in df_na_filtered.columns:
    if df_na_filtered[column].dtype == 'object':
        categoical_columns.append(column)
    else:
        numerical_columns.append(column)

data_frame_for_prediction = df_na_filtered.iloc[:, :].copy(deep=True)
#You could choose to use either label encoder or one hot encoding
data_frame_for_prediction = pd.get_dummies(data_frame_for_prediction, columns=categoical_columns)
data_frame_for_prediction.head()

<h1> Data Engineering </h1>

In [ ]:
#lets start with prices
red_square = dict(markerfacecolor='r', markeredgecolor='r', marker='.')
data_frame_for_prediction['total_price'].plot(kind='box', xlim=(0, 1000), vert=False, flierprops=red_square, figsize=(16,2))
df_prices = data_frame_for_prediction[ (data_frame_for_prediction.total_price > 1000) | (data_frame_for_prediction.total_price == 0) ]
print(len(df_prices))
print(data_frame_for_prediction['total_price'].describe())

In [ ]:
#we will drop all the columns that are going to be beyond 2500 unit price range, this is an feature engineering test and can be optinally tested
data_frame_for_prediction.drop(data_frame_for_prediction[ (data_frame_for_prediction.total_price > 1000) | (data_frame_for_prediction.total_price == 0) ].index, axis=0, inplace=True)

In [ ]:
plt.figure(1)
f, axarr = plt.subplots(3, 2, figsize=(10, 9))
sns.scatterplot(x='host_is_superhost', y= 'total_price', data = data_frame_for_prediction,ax= axarr[0, 0])
sns.scatterplot(x='host_identity_verified', y= 'total_price', data = data_frame_for_prediction,ax= axarr[0, 1])
sns.scatterplot(x='requires_license', y= 'total_price', data = data_frame_for_prediction,ax= axarr[1, 0])
sns.scatterplot(x='require_guest_profile_picture', y= 'total_price', data = data_frame_for_prediction,ax= axarr[1, 1])
sns.scatterplot(x='require_guest_phone_verification', y= 'total_price', data = data_frame_for_prediction,ax= axarr[2, 0])
sns.scatterplot(x='host_has_profile_pic', y= 'total_price', data = data_frame_for_prediction,ax= axarr[2, 1])
plt.tight_layout()
plt.show()

In [ ]:
data_frame_for_prediction.drop(["host_is_superhost",
                                "host_identity_verified", 
                                "require_guest_profile_picture",
                                "require_guest_phone_verification"], inplace=True, axis=1)

In [ ]:
#Dropping the amenities and check the prediction
df_without_ammenities = data_frame_for_prediction.drop(amenities_columns, inplace=False, axis=1).copy(deep=True)

In [ ]:
df_without_ammenities.head()

In [ ]:
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import accuracy_score
from sklearn.linear_model import Ridge
import xgboost as xgb


<h1> Model Training and Validation using K-Fold Cross Validation</h1>

In [ ]:
def print_performance(accuracy_model_train, accuracy_model_test, K):
    avg_r = 0;
    avg_rmse = 0;
    print("\nTraining:")
    for (r, rmse) in accuracy_model_train:
        avg_r = avg_r + r
        avg_rmse = avg_rmse + rmse
    print("\nR2 = {}  RMSE= {}".format(round(avg_r/K, 3), round(avg_rmse/K, 3)))

    avg_r = 0;
    avg_rmse = 0;    
    print("\nTesting:")    
    for (r, rmse) in accuracy_model_test:
        avg_r = avg_r + r;
        avg_rmse = avg_rmse + rmse

    print("\nR2 = {}  RMSE= {}".format(round(avg_r/K, 3), round(avg_rmse/K, 3)))
    

In [ ]:
#K-fold Validation :
def train_model(model, b_transform):
    df_kfold = data_frame_for_prediction.copy(deep=True)

    #columns in prediction :
    y = df_kfold['total_price'].copy(deep=True)
    df_kfold.drop(['total_price'], inplace=True, axis=1)
    X =  df_kfold

    #scale all the values
    sc = StandardScaler()
    K = 10
    kf = KFold(n_splits = K, random_state=None, shuffle=True)
    iterationCount = 1
    accuracy_model_train = []
    accuracy_model_test = []

    for train_index, test_index in kf.split(X):
                print("\n Fold ", iterationCount)
                #print("TRAIN:", train_index, "TEST:", test_index)
                X_train, X_test =  X.iloc[train_index,:], X.iloc[test_index,:]
                y_train, y_test = y.iloc[train_index], y.iloc[test_index]
                if b_transform == 1:
                    X_train = sc.fit_transform(X_train)
                    X_test  = sc.fit_transform(X_test)
                model.fit(X_train, y_train) 

                y_pred_train = model.predict(X_train)
                r2_train = r2_score(y_train, y_pred_train)

                y_pred_test = model.predict(X_test)
                r2_test = r2_score(y_test, y_pred_test)

                accuracy_model_train.append([r2_train, mean_squared_error(y_train, y_pred_train,  squared=False)])
                accuracy_model_test.append([r2_test, mean_squared_error(y_test, y_pred_test,  squared=False)])
                
                iterationCount = iterationCount + 1   
    print_performance(accuracy_model_train, accuracy_model_test, K)

In [ ]:
print("\n***********************************\n")
print("\nLinear Regression\n")
model =  LinearRegression()
train_model(model, 0)

In [ ]:

print("\n***********************************\n")
print("\nRidge Regression\n")
model =  Ridge()
train_model(model, 1)


In [1]:

print("\n***********************************\n")
print("\nRandom Forest Regressor\n")
model =  RandomForestRegressor(max_depth=5,verbose=1,random_state=0,n_estimators=200,n_jobs=-1)
train_model(model, 1)



***********************************


Random Forest Regressor



NameError: name 'RandomForestRegressor' is not defined

In [ ]:

print("\n***********************************\n")
print("\nXG Boost - XGBRegressor\n")
# instantiate xgboost with best parameters
booster = xgb.XGBRegressor(colsample_bytree=0.7, gamma=0.2, learning_rate=0.1, 
                           max_depth=6, n_estimators=200, random_state=4,silent=True)

train_model(booster, 1)

In [ ]:
#Prepare the Training Data:
from sklearn.preprocessing import StandardScaler
#columns in prediction : 
columns_in_prediction = [col for col in data_frame_for_prediction.columns if col != "total_price"]
X_train, X_test, y_train, y_test = train_test_split(data_frame_for_prediction[columns_in_prediction], data_frame_for_prediction['total_price'], test_size=0.2)

#scale all the values
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test  = sc.transform(X_test)

print(len(data_frame_for_prediction.columns))

In [ ]:
import xgboost as xgb

# Fitting the model
xgb_reg = xgb.XGBRegressor(silent=True)
xgb_reg.fit(X_train, y_train)
training_preds_xgb_reg = xgb_reg.predict(X_train)
val_preds_xgb_reg = xgb_reg.predict(X_test)

# Printing the results
print("\nTraining RMSE:", round(mean_squared_error(y_train, training_preds_xgb_reg, squared=False),2))
print("Validation RMSE:", round(mean_squared_error(y_test, val_preds_xgb_reg, squared=False),2))
print("\nTraining r2:", round(r2_score(y_train, training_preds_xgb_reg),2))
print("Validation r2:", round(r2_score(y_test, val_preds_xgb_reg),2))

# Producing a dataframe of feature importances
ft_weights_xgb_reg = pd.DataFrame(xgb_reg.feature_importances_, columns=['total_price'], index=columns_in_prediction)
ft_weights_xgb_reg.sort_values('total_price', inplace=True)

# Plotting feature importances
plt.figure(figsize=(8,8))
plt.barh(ft_weights_xgb_reg.index, ft_weights_xgb_reg['total_price'], align='center') 
plt.title("Feature importances in the XGBoost model", fontsize=14)
plt.xlabel("Feature importance")
plt.margins(y=0.01)
plt.show()

<h1> Heavy Running Code </h1>

In [ ]:
from sklearn.model_selection import GridSearchCV
booster = xgb.XGBRegressor()
# create Grid
param_grid = {'n_estimators': [100, 150, 200],
              'learning_rate': [0.01, 0.05, 0.1], 
              'max_depth': [3, 4, 5, 6, 7],
              'colsample_bytree': [0.6, 0.7, 1],
              'gamma': [0.0, 0.1, 0.2]}

# instantiate the tuned random forest
booster_grid_search = GridSearchCV(booster, param_grid, cv=3, n_jobs=-1)

# train the tuned random forest
booster_grid_search.fit(X_train, y_train)

# print best estimator parameters found during the grid search
print(booster_grid_search.best_params_)

In [ ]:
# instantiate xgboost with best parameters
booster = xgb.XGBRegressor(colsample_bytree=0.7, gamma=0.2, learning_rate=0.1, 
                           max_depth=6, n_estimators=200, random_state=4,slient=True)

# train
booster.fit(X_train, y_train)

# predict
y_pred_train = booster.predict(X_train)
y_pred_test = booster.predict(X_test)

r2 = r2_score(y_test, y_pred_test)
print(f"r2: {round(r2, 4)}")
rmse = mean_squared_error(y_test, y_pred_test)
print(f"RMSE: {round(np.sqrt(rmse), 4)}")